In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from math import sqrt
import joblib
from projectwind.LSTM_weather_forecast import get_LSTM_data, WindowGenerator
from google.cloud import storage
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf


In [3]:
train_df, val_df, test_df = get_LSTM_data()

### Fetching 25xWTG data ###
### Fetching weather API data ###
### Preparing datasets ###


In [5]:
# Create window
n_steps_in = 3 * 24   # hrs
n_steps_out = 6   # hrs
window = WindowGenerator(input_width=n_steps_in, label_width=n_steps_out, shift=n_steps_out,
                         train_df=train_df, val_df=val_df, test_df=test_df,
                         input_columns=[]
                         forecast_columns=['windSpeed_API','windGust_API',
                                        'Wind_API_X', 'Wind_API_Y',
                                        'WindGust_API_X','WindGust_API_Y'],
                         label_columns=['Power'])


In [16]:
train_df[0].columns

Index(['Power', 'Rotor Speed', 'Wind Speed', 'Blade Pitch', 'Nacelle_X',
       'Nacelle_Y', 'Wind_X', 'Wind_Y', 'windSpeed_API', 'windGust_API',
       'Wind_API_X', 'Wind_API_Y', 'WindGust_API_X', 'WindGust_API_Y'],
      dtype='object')

In [15]:
# Fetch test data
X_test,  X_fc_test,  y_test  =  window.test

#Load model
model = tf.keras.models.load_model('checkpoints/Oracle_model/LSTM_Forecast_Hybrid_num_3Lx16N_fc_2Lx54-18N_2x02do.h5')

y_pred = model.predict([X_test, X_fc_test], batch_size=1)

ValueError: in user code:

    File "/Users/elliotbettane/.pyenv/versions/3.8.11/envs/projectwind/lib/python3.8/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/Users/elliotbettane/.pyenv/versions/3.8.11/envs/projectwind/lib/python3.8/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/elliotbettane/.pyenv/versions/3.8.11/envs/projectwind/lib/python3.8/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/elliotbettane/.pyenv/versions/3.8.11/envs/projectwind/lib/python3.8/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/Users/elliotbettane/.pyenv/versions/3.8.11/envs/projectwind/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_

In [ ]:
def download_model(model_directory="PipelineTest", bucket=BUCKET_NAME, rm=True):
    client = storage.Client().bucket(bucket)

    storage_location = 'models/{}/versions/{}/{}'.format(
        model,
        model_directory,
        'model.joblib')
    blob = client.blob(storage_location)
    blob.download_to_filename('model.joblib')
    print("=> pipeline downloaded from storage")
    model = joblib.load('model.joblib')
    if rm:
        os.remove('model.joblib')
    return model

In [ ]:
def get_model(path_to_joblib):
    pipeline = joblib.load(path_to_joblib)
    return pipeline


In [ ]:
def evaluate_model(y, y_pred):
    MAE = round(mean_absolute_error(y, y_pred), 2)
    RMSE = round(sqrt(mean_squared_error(y, y_pred)), 2)
    res = {'MAE': MAE, 'RMSE': RMSE}
    return res